# Problem Statement

## Q1. Linear Programming

Your start-up will face the cash requirements shown in Table 1 in the next eight quarters (positive entries represent cash needs while negative entries represent cash surpluses). The company has three borrowing possibilities.

**Table 1: Cash Flow (in Crores of INR)**

| Q1  | Q2  | Q3  | Q4   | Q5   | Q6  | Q7  | Q8   |
| --- | --- | --- | ---- | ---- | --- | --- | ---- |
| 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

**Borrowing Possibilities:**

*   A 2-year loan available at the beginning of Q1, with a 1% interest per quarter.
*   The other two borrowing opportunities are available at the beginning of every quarter:
    *   A 6-month loan with a 1.8% interest per quarter.
    *   A quarterly loan with a 2.5% interest for the quarter.
*   Any surplus can be invested at a 0.5% interest per quarter.

**Questions:**

a) Formulate a LP that maximizes the wealth of the company at the beginning of Q9.

b) Use built-in LP solvers to get a solution.

c) What is a good initial basic feasible point?

d) Use the revised simplex method and see if your solution matches the one from built-in solver. Explain.

# My approach
## Assumptions
- <span style="color:blue">We have to pay the costs of the quarter before our investment for the quarter matured.</span>
- <span style="color:blue">We pay compound interest with the principal at the end of the loan term</span>
- <span style="color:blue">We cannot take a 6 month loan at the beginning of the 8th quarter as this will be repayed after the ninth quarter</span>
- <span style="color:blue">The interest on out investments is available before we have to repay our loans</span>
- <span style="color:blue">Fresh loans can be raised only after the interest payment of existing loans.</span>

## Variables

- $x_t$: Total money at the start of quarter $t$ **after** receiving loans  
- $L$: Two-year loan available at the beginning of the first quarter  
- $j_t$: 6-month loan available at the beginning of each quarter $t$  
- $k_t$: 4-month loan available at the beginning of each quarter $t$  
- $a_t$: Total money at the start of quarter $t$ **before** receiving loans, after all costs and loan repayments from the previous quarter  
- $i_t$: Investment made in quarter $t$  
- $c_t$: Cash flow in quarter $t$ provided in the table 
- $r$: Final result (e.g., total return) to be maximized


## Constraints

### Non-negativity constraints
All variables must be non-negative:

- $a_t \geq 0$, $x_t \geq 0$, $j_t \geq 0$, $k_t \geq 0$, $i_t \geq 0$

All $c_t$ values must match the given values

### Initial condition
- $a_1 = 0$

### Quarter 1 constraints
- $x_1 = L + j_1 + k_1 + a_1$
- $i_1 \leq x_1$
- $c_1 \leq x_1 - i_1$
- $a_2 = x_1 - c_1 + 0.005 \cdot i_1   - 1.025 \cdot k_1$

*(Note: This expression assumes interest from investments is available before repayment. If loans must be repaid before interest is applied, we require an additional non-negativity constraint.)*

### General constraints (for $t = 2$ to $8$)
- $x_t = a_t + j_t + k_t$
- $i_t \leq x_t$
- $c_t  \leq x_t - i_t$  
- $a_{t+1} = x_t + 0.005 \cdot i_t  - c_t - (1+0.018)^2 \cdot j_{t-1}   - 1.025 \cdot k_t$

### Final quarter constraint
- $j_8 = 0$  
  *(No new 6-month loan in the final quarter)*

### Objective
- $r = a_9 - (1+0.01)^8 \cdot L$  
  *(Maximize $r$, the net return after repaying the two-year loan)*


In [6]:
import pulp as pl

# Create the LP problem
model = pl.LpProblem("Cash_Flow_Optimization", pl.LpMaximize)

# Define the variables
L = pl.LpVariable("L", lowBound=0)  # 2-year loan at beginning of Q1

# Lists to store variables for each quarter (use 1-based indexing to match the problem)
a = [None]  # Money at start of quarter before receiving loans
x = [None]  # Total money at start of quarter after loans
i = [None]  # Investment in quarter
j = [None]  # 6-month loan
k = [None]  # Quarterly loan

# Create variables for quarters 1-8
for t in range(1, 9):
    a.append(pl.LpVariable(f"a_{t}", lowBound=0))
    x.append(pl.LpVariable(f"x_{t}", lowBound=0))
    i.append(pl.LpVariable(f"i_{t}", lowBound=0))
    j.append(pl.LpVariable(f"j_{t}", lowBound=0))
    k.append(pl.LpVariable(f"k_{t}", lowBound=0))

# Add a_9 for final wealth
a.append(pl.LpVariable("a_9", lowBound=0))

# Define cash flows as given in the problem
c = [None, 100, 500, 100, -600, -500, 200, 600, -900]

# Define the objective variable
r = pl.LpVariable("r", None)  # Final wealth to maximize

# Add objective to the model
model += r

# Initial condition
model += a[1] == 0, "Initial_money"

# No 6-month loan in last quarter
model += j[8] == 0, "No_6month_loan_in_Q8"

# Variable r definition (final wealth after repaying 2-year loan)
model += r == a[9] - (1 + 0.01)**8 * L, "Final_wealth"

# Add constraints for each quarter
for t in range(1, 9):
    # Money after receiving loans
    model += x[t] == a[t] + j[t] + k[t] + (L if t == 1 else 0), f"Money_after_loans_in_Q{t}"
    
    # Investment constraint
    model += i[t] <= x[t], f"Investment_constraint_in_Q{t}"
    
    # Ensure enough money for cash flow requirement
    model += x[t] - i[t] >= c[t], f"Cash_flow_constraint_in_Q{t}"
    
    # Money balance equation for next quarter
    if t==1:
        model += a[t+1] == x[t] - c[t] + 0.005 * i[t] -1.025*k[t], f"Balance_equation_in_Q{t}"
    else:
        model += a[t+1] == x[t] - c[t] + 0.005 * i[t] - 1.025 * k[t] - (1+0.018)**2 * j[t-1], f"Balance_equation_in_Q{t}"

# Solve the model
model.solve()

print(f"Status: {pl.LpStatus[model.status]}")
print(f"Optimal wealth at Q9: {pl.value(r)}")

# Print results
print("\nOptimal decisions:")
print(f"2-year loan at Q1: {pl.value(L):.2f}")
print("\nQuarterly values:")
print("Quarter | Cash Flow | 6-month Loan | Quarterly Loan | Investment | Balance after loans")
print("--------|-----------|--------------|----------------|------------|------------------")

for t in range(1, 9):
    print(f"Q{t}      | {c[t]:9.2f} | {pl.value(j[t]):12.2f} | {pl.value(k[t]):14.2f} | {pl.value(i[t]):10.2f} | {pl.value(x[t]):18.2f}")

print(f"\nFinal wealth at beginning of Q9: {pl.value(r):.2f}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/harshit/anaconda3/envs/ml/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/098a6d8017dc4e25992d12ab3af8cb2e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/098a6d8017dc4e25992d12ab3af8cb2e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 40 COLUMNS
At line 151 RHS
At line 187 BOUNDS
At line 189 ENDATA
Problem MODEL has 35 rows, 43 columns and 109 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 31 (-4) rows, 39 (-4) columns and 101 (-8) elements
Perturbing problem by 0.001% of 2.0999054 - largest nonzero change 1.0435747e-06 ( 0.00035544633%) - largest zero change 1.0410382e-06
0  Obj -0.0044514761 Primal inf 33249.541 (18) Dual inf 0.27190931 (1)
0  Obj -0 Primal inf 33249.541 (18) Dual inf 5.3933194e+11 (15)
27  Obj 457.85684
Optimal - objective value 457.85684